# 🌳 Tutorial Completo: Árbol de Decisión para Clasificación

Este notebook contiene un flujo completo de Machine Learning usando **Árboles de Decisión** para clasificación binaria.

## 📋 Contenido:
1. Importación de librerías
2. Carga y preparación de datos
3. Preprocesamiento (opcional según tu dataset)
4. División de datos
5. Entrenamiento del modelo
6. Optimización de hiperparámetros
7. Evaluación y visualizaciones
8. Análisis avanzado

---
## 1️⃣ PASO 1: Importar Librerías Necesarias

**¿Qué hace?** Importa todas las librerías que usaremos durante el análisis.

**Obligatorio:** ✅ SÍ - Siempre necesario al inicio.

In [ ]:
# Librerías básicas para manejo de datos
import numpy as np
import pandas as pd

# Librerías de scikit-learn para el modelo y división de datos
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Librerías para evaluación y métricas
from sklearn.metrics import (ConfusionMatrixDisplay, RocCurveDisplay,
                             PrecisionRecallDisplay, classification_report)

---
## 2️⃣ PASO 2: Cargar y Preparar el Dataset

**¿Qué hace?** 
- Lee el archivo CSV con los datos
- Separa las **variables independientes (X)** de la **variable objetivo (y)**

**Parámetros a modificar:**
- `"diabetes.csv"` → Cambia esto por el nombre de tu archivo
- `"Outcome"` → Cambia esto por el nombre de tu columna objetivo

**Obligatorio:** ✅ SÍ - Necesitas cargar tus datos.

In [ ]:
# LEER ARCHIVO CSV
# 📝 MODIFICAR: Cambia "diabetes.csv" por el nombre de tu archivo
df = pd.read_csv("diabetes.csv")

# IDENTIFICAR COLUMNA OBJETIVO (la que queremos predecir)
# 📝 MODIFICAR: Cambia "Outcome" por el nombre de tu columna objetivo
y = df["Outcome"]  # Variable dependiente (lo que queremos predecir)

# ELIMINAR COLUMNA OBJETIVO del conjunto de características
X = df.drop(columns=["Outcome"])  # Variables independientes (features)

# MOSTRAR LA SEPARACIÓN para verificar
display(y.head())  # Primeros valores de la variable objetivo
display(X.head())  # Primeras filas de las características

### 📌 MÉTODO ALTERNATIVO: Selección por Índices

**Obligatorio:** ❌ NO - Solo si prefieres seleccionar columnas por posición en lugar de por nombre.

**¿Cuándo usar?** Si no tienes nombres de columnas o prefieres trabajar con índices numéricos.

In [ ]:
# OTRA FORMA DE OBTENER VARIABLES (usando índices)
# X = df.iloc[:, 0:8].values   # Toma columnas 0 a 7 (todas las características)
# y = df.iloc[:, -1].values     # Toma la última columna (objetivo)

# 📝 MODIFICAR:
# - 0:8 → Cambia por el rango de tus columnas de características
# - -1 → Mantener si tu objetivo está en la última columna, sino cambia el índice

---
## 3️⃣ PASO 3: Codificación de Variables Categóricas (Variables Dummies)

**Obligatorio:** ❌ NO - **SOLO si tienes variables categóricas (texto) en tu dataset**

**¿Cuándo usar?**
- Si tienes columnas con valores como: "Masculino/Femenino", "Alto/Medio/Bajo", etc.
- Si todas tus columnas ya son numéricas, **SALTA ESTA CELDA**

**⚠️ ADVERTENCIA:** El código actual tiene errores. Aquí está la versión corregida.

**Parámetros:**
- `[3]` → Índice de la columna categórica a convertir (cambia según tu dataset)

In [ ]:
# ⚠️ ESTA CELDA ES OPCIONAL - Solo si tienes variables categóricas

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer

# Supongamos que la columna 3 es categórica (ej: "Alto", "Medio", "Bajo")
# 📝 MODIFICAR: Cambia [3] por el índice de tu columna categórica

# Convertir X a numpy array si es DataFrame
X_array = X.values

# Codificar la columna categórica a números
labelencoder_X = LabelEncoder()
X_array[:, 3] = labelencoder_X.fit_transform(X_array[:, 3])

# Aplicar One-Hot Encoding (crear variables dummy)
onehotencoder = make_column_transformer(
    (OneHotEncoder(), [3]),  # Columna a codificar
    remainder="passthrough"  # Mantener las demás columnas sin cambios
)
X = onehotencoder.fit_transform(X_array)

# ELIMINAR la primera columna dummy para evitar multicolinealidad (Dummy Variable Trap)
X = X[:, 1:]

---
## 4️⃣ PASO 4: Escalado de Variables (Normalización/Estandarización)

**Obligatorio:** ⚠️ DEPENDE

**¿Cuándo usar?**
- **SÍ necesario** para: SVM, KNN, Regresión Logística, Redes Neuronales
- **NO necesario** para: Árboles de Decisión, Random Forest, XGBoost

Para **Árboles de Decisión** (nuestro caso) puedes **SALTAR** esta celda, pero no hace daño aplicarlo.

**¿Qué hace?** Escala las variables para que tengan media 0 y desviación estándar 1.

In [ ]:
# ⚠️ ESTA CELDA ES OPCIONAL para Árboles de Decisión

from sklearn.preprocessing import StandardScaler

# Crear el escalador
scaler = StandardScaler()

# Ajustar y transformar los datos
X = scaler.fit_transform(X)

# Verificar el resultado
print("Datos escalados (primeras 5 filas):")
print(X[:5])

---
## 5️⃣ PASO 5: Dividir los Datos en Entrenamiento y Prueba

**Obligatorio:** ✅ SÍ - Fundamental para evaluar el modelo correctamente.

**Parámetros importantes:**

### 🔧 `test_size` (Tamaño del conjunto de prueba)
- **Valor:** 0.25 (25% prueba, 75% entrenamiento)
- **Recomendaciones según tamaño del dataset:**
  - Dataset GRANDE (>10,000 filas): `0.10` a `0.20` (10-20%)
  - Dataset MEDIANO (1,000-10,000): `0.20` a `0.30` (20-30%)
  - Dataset PEQUEÑO (<1,000): `0.30` a `0.40` (30-40%)

### 🔧 `random_state`
- **Valor:** 42 (puede ser cualquier número)
- **Propósito:** Hace que la división sea reproducible (siempre igual)

### 🔧 `stratify`
- **Valor:** `y` (la variable objetivo)
- **Propósito:** Mantiene la misma proporción de clases en entrenamiento y prueba
- **Ejemplo:** Si tienes 60% clase 0 y 40% clase 1, ambos conjuntos tendrán esa proporción

### 🔧 `shuffle` (no usado aquí, pero disponible)
- **Valor por defecto:** `True`
- **Propósito:** Mezcla los datos antes de dividir
- **Cuándo usar False:** Solo si tus datos tienen orden temporal importante

In [ ]:
# DIVIDIR LA DATA en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,      # 📝 MODIFICAR: 25% para prueba, 75% para entrenar
    random_state=42,     # 📝 MODIFICAR: Cualquier número para reproducibilidad
    stratify=y           # Mantiene proporciones de clases balanceadas
)

print(f"Datos de entrenamiento: {X_train.shape[0]} filas")
print(f"Datos de prueba: {X_test.shape[0]} filas")
print(f"\nDistribución de clases en entrenamiento:")
print(y_train.value_counts(normalize=True))
print(f"\nDistribución de clases en prueba:")
print(y_test.value_counts(normalize=True))

---
## 6️⃣ PASO 6: Crear y Entrenar el Modelo (Versión Básica)

**Obligatorio:** ✅ SÍ - Este es el corazón del análisis.

**Parámetros del DecisionTreeClassifier:**

### 🔧 `criterion` (Criterio de división)
- **Opciones:** `"gini"`, `"entropy"`, `"log_loss"`
- **Por defecto:** `"gini"`
- **Gini:** Más rápido, usado comúnmente
- **Entropy:** Basado en teoría de información, puede ser más preciso
- **Log_loss:** Similar a entropy, útil en algunos casos

### 🔧 `max_depth` (Profundidad máxima del árbol)
- **Opciones:** Cualquier entero positivo o `None`
- **Por defecto:** `None` (sin límite)
- **Recomendado:** 3-10 para evitar sobreajuste
- **Menor profundidad:** Modelo más simple, menos sobreajuste
- **Mayor profundidad:** Modelo más complejo, puede sobreajustar

### 🔧 `random_state` (Semilla aleatoria)
- **Valor:** Cualquier entero
- **Propósito:** Reproducibilidad de resultados

In [ ]:
# CREAR EL MODELO DE ÁRBOL DE DECISIÓN (versión básica con parámetros por defecto)
tree = DecisionTreeClassifier()

# Parámetros opcionales que puedes agregar:
# criterion="gini",        # 📝 MODIFICAR: "gini", "entropy", o "log_loss"
# max_depth=4,             # 📝 MODIFICAR: Número entre 2-10, o None para sin límite
# random_state=42          # 📝 MODIFICAR: Para reproducibilidad

print("Modelo creado con parámetros por defecto")

In [ ]:
# ENTRENAR EL MODELO con los datos de entrenamiento
tree.fit(X_train, y_train)

print("✅ Modelo entrenado exitosamente")
print(f"Profundidad del árbol: {tree.get_depth()}")
print(f"Número de hojas: {tree.get_n_leaves()}")

In [ ]:
# HACER PREDICCIONES con los datos de prueba
y_pred = tree.predict(X_test)

print("Predicciones realizadas:")
print(y_pred[:10])  # Mostrar las primeras 10 predicciones

In [ ]:
# EVALUAR LA PRECISIÓN del modelo
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"📊 Precisión del modelo: {accuracy:.4f} ({accuracy*100:.2f}%)")

---
## 7️⃣ PASO 7: Optimización de Hiperparámetros con GridSearchCV

**Obligatorio:** ❌ NO - Pero **MUY RECOMENDADO** para mejorar el rendimiento.

**¿Qué hace?** Prueba todas las combinaciones de parámetros para encontrar la mejor.

**Parámetros de GridSearchCV:**

### 🔧 `param_grid` (Diccionario de parámetros a probar)
- Define qué valores probar para cada parámetro
- **Más opciones = Más tiempo de ejecución**

### 🔧 `cv` (Cross-validation)
- **Valor:** Número de "folds" (divisiones)
- **Común:** 5 o 10
- **Mayor valor:** Más preciso pero más lento

### 🔧 `n_jobs`
- **Valor:** -1 (usar todos los procesadores)
- **Propósito:** Acelerar el proceso usando paralelización

### 🔧 `scoring`
- **Opciones:** "accuracy", "f1", "precision", "recall", "roc_auc"
- **Por defecto:** "accuracy"

In [ ]:
# DEFINIR PARÁMETROS A PROBAR
param_dist = {
    "criterion": ["gini", "entropy", "log_loss"],  # 📝 MODIFICAR: Criterios a probar
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, None],  # 📝 MODIFICAR: Profundidades a probar
}

# NOTA: Puedes agregar más parámetros:
# "min_samples_split": [2, 5, 10],      # Mínimo de muestras para dividir un nodo
# "min_samples_leaf": [1, 2, 5],        # Mínimo de muestras en cada hoja
# "max_features": [None, "sqrt", "log2"] # Número máximo de características a considerar

print(f"Se probarán {len(param_dist['criterion']) * len(param_dist['max_depth'])} combinaciones")

In [ ]:
# CREAR Y EJECUTAR GRID SEARCH
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    tree,
    param_grid=param_dist,
    cv=15,              # 📝 MODIFICAR: Número de folds (común: 5 o 10)
    n_jobs=-1,          # Usar todos los procesadores disponibles
    scoring="accuracy"  # 📝 MODIFICAR: Métrica a optimizar
)

print("⏳ Buscando mejores parámetros... (esto puede tardar)")
grid.fit(X_train, y_train)
print("✅ Búsqueda completada")

In [ ]:
# VER EL MEJOR ESTIMADOR ENCONTRADO
print("🏆 Mejor modelo encontrado:")
print(grid.best_estimator_)

In [ ]:
# VER EL MEJOR SCORE (precisión)
print(f"📊 Mejor score (accuracy): {grid.best_score_:.4f}")

In [ ]:
# VER LOS MEJORES PARÁMETROS
print("⚙️ Mejores parámetros encontrados:")
for param, value in grid.best_params_.items():
    print(f"  • {param}: {value}")

---
## 8️⃣ PASO 8: Entrenar Modelo Final con Mejores Parámetros

**Obligatorio:** ✅ SÍ - Si hiciste GridSearch, usa los mejores parámetros encontrados.

**¿Qué hace?** Crea un nuevo modelo con los parámetros óptimos y lo evalúa.

In [ ]:
# CREAR MODELO CON LOS MEJORES PARÁMETROS
# 📝 MODIFICAR: Usa los valores que obteniste de grid.best_params_

tree_optimized = DecisionTreeClassifier(
    criterion="gini",    # 📝 Usar el valor de best_params_
    max_depth=8,         # 📝 Usar el valor de best_params_
    random_state=42
)

# ENTRENAR
tree_optimized.fit(X_train, y_train)

# PREDECIR
y_pred_optimized = tree_optimized.predict(X_test)

# EVALUAR
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
print(f"📊 Precisión del modelo optimizado: {accuracy_optimized:.4f} ({accuracy_optimized*100:.2f}%)")

---
## 9️⃣ PASO 9: Visualización de Métricas - Matriz de Confusión

**Obligatorio:** ⚠️ RECOMENDADO - Esencial para entender el rendimiento del modelo.

**¿Qué muestra?**
- **Verdaderos Positivos (VP):** Predicciones correctas de clase 1
- **Verdaderos Negativos (VN):** Predicciones correctas de clase 0
- **Falsos Positivos (FP):** Predicciones incorrectas como clase 1
- **Falsos Negativos (FN):** Predicciones incorrectas como clase 0

In [ ]:
import matplotlib.pyplot as plt

# MATRIZ DE CONFUSIÓN
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
ConfusionMatrixDisplay.from_estimator(
    tree_optimized,
    X_test,
    y_test,
    ax=ax,
    cmap="Blues"  # 📝 MODIFICAR: Color ("Blues", "Greens", "Reds", "Purples")
)
ax.set_title("Matriz de Confusión — Árbol de Decisión Optimizado")
plt.tight_layout()
plt.show()

# REPORTE DE CLASIFICACIÓN (métricas detalladas)
print("\n📋 Reporte de Clasificación:")
print(classification_report(y_test, y_pred_optimized))

**📖 Interpretación del Reporte:**
- **Precision:** De todas las predicciones positivas, ¿cuántas fueron correctas?
- **Recall (Sensibilidad):** De todos los positivos reales, ¿cuántos detectamos?
- **F1-Score:** Media armónica de Precision y Recall
- **Support:** Número de muestras de cada clase

---
## 🔟 PASO 10: Curvas ROC y Precision-Recall

**Obligatorio:** ⚠️ RECOMENDADO - Muy útil para evaluar modelos de clasificación.

**Curva ROC:**
- Muestra la relación entre Tasa de Verdaderos Positivos vs Falsos Positivos
- **AUC (Area Under Curve):** Métrica resumen (0.5 = aleatorio, 1.0 = perfecto)

**Curva Precision-Recall:**
- Útil cuando las clases están desbalanceadas
- Muestra el balance entre Precision y Recall

In [ ]:
# CURVA ROC
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
RocCurveDisplay.from_estimator(tree_optimized, X_test, y_test, ax=ax)
ax.set_title("Curva ROC")
ax.plot([0, 1], [0, 1], 'k--', label='Clasificador Aleatorio')  # Línea diagonal
ax.legend()
plt.tight_layout()
plt.show()

# CURVA PRECISION-RECALL
fig, ax = plt.subplots(figsize=(5, 4), dpi=120)
PrecisionRecallDisplay.from_estimator(tree_optimized, X_test, y_test, ax=ax)
ax.set_title("Curva Precision–Recall")
plt.tight_layout()
plt.show()

---
## 1️⃣1️⃣ PASO 11: Importancia de Variables

**Obligatorio:** ❌ NO - Pero muy útil para interpretar el modelo.

**¿Qué muestra?** Qué características (variables) son más importantes para las predicciones del árbol.

**⚠️ NOTA:** Esta celda requiere que `X` sea un DataFrame con nombres de columnas.

In [ ]:
# IMPORTANCIA DE VARIABLES
importances = tree_optimized.feature_importances_

# Obtener nombres de características (ajustar según tu caso)
if isinstance(X, pd.DataFrame):
    feat_names = X.columns.to_numpy()
else:
    # Si X no es DataFrame, crear nombres genéricos
    feat_names = np.array([f"Feature_{i}" for i in range(X.shape[1])])

# Ordenar por importancia (de mayor a menor)
order = np.argsort(importances)[::-1]

# GRAFICAR
fig, ax = plt.subplots(figsize=(6, 4), dpi=120)
ax.bar(range(len(importances)), importances[order])
ax.set_xticks(range(len(importances)))
ax.set_xticklabels(feat_names[order], rotation=45, ha="right")
ax.set_ylabel("Importancia")
ax.set_title("Importancia de Variables — Árbol de Decisión")
plt.tight_layout()
plt.show()

# MOSTRAR TABLA
print("\n📊 Ranking de Importancia:")
importance_df = pd.DataFrame({
    'Variable': feat_names[order],
    'Importancia': importances[order]
})
print(importance_df)

---
## 1️⃣2️⃣ PASO 12: Visualizar el Árbol de Decisión

**Obligatorio:** ❌ NO - Pero excelente para entender cómo funciona el modelo.

**Parámetros de plot_tree:**
- **`filled=True`:** Colorea los nodos según la clase predominante
- **`rounded=True`:** Bordes redondeados (más estético)
- **`impurity=True`:** Muestra el valor de Gini/Entropy en cada nodo
- **`proportion=True`:** Muestra proporciones en lugar de conteos absolutos

In [ ]:
from sklearn import tree as sktree

# Obtener nombres de características y clases
if isinstance(X, pd.DataFrame):
    feature_names = X.columns.tolist()
else:
    feature_names = [f"Feature_{i}" for i in range(X.shape[1])]

class_names = [str(c) for c in np.unique(y)]

# VISUALIZAR ÁRBOL
fig, ax = plt.subplots(figsize=(14, 7), dpi=120)  # 📝 MODIFICAR: Ajustar tamaño si es necesario

sktree.plot_tree(
    tree_optimized,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,          # Colorear nodos
    rounded=True,         # Bordes redondeados
    impurity=True,        # Mostrar Gini/Entropy
    proportion=True       # Mostrar proporciones
)
ax.set_title("Árbol de Decisión Optimizado")
plt.show()

---
## 1️⃣3️⃣ PASO 13: Exportar Árbol como Imagen de Alta Calidad (Graphviz)

**Obligatorio:** ❌ NO - Solo si quieres guardar una imagen del árbol.

**⚠️ REQUISITO:** Solo funciona en Google Colab o si tienes Graphviz instalado localmente.

**¿Cuándo usar?**
- Si estás en **Google Colab** → ✅ Usar
- Si estás en **Jupyter local** → Solo si instalaste Graphviz
- Si no funciona → Usa el método anterior (plot_tree)

In [ ]:
# ⚠️ SOLO EN GOOGLE COLAB - Instalar Graphviz
# Descomentar las siguientes líneas si estás en Colab:

# !apt-get -qq install graphviz
# !pip -q install graphviz pydotplus

In [ ]:
# EXPORTAR ÁRBOL CON GRAPHVIZ (versión de alta calidad)
from sklearn.tree import export_graphviz
import graphviz

# Obtener nombres
if isinstance(X, pd.DataFrame):
    feature_names = X.columns.tolist()
else:
    feature_names = [f"Feature_{i}" for i in range(X.shape[1])]

class_names = [str(c) for c in np.unique(y)]

# GENERAR GRÁFICO
dot = export_graphviz(
    tree_optimized,
    out_file=None,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,
    rounded=True,
    special_characters=True,
    impurity=True,
    proportion=True
)

graph = graphviz.Source(dot)
graph.format = "png"  # 📝 MODIFICAR: "png", "svg", "pdf"
graph.render("arbol_decision", cleanup=True)  # Guarda como archivo

# Mostrar en notebook
graph

---
## 1️⃣4️⃣ PASO 14: Curva de Aprendizaje

**Obligatorio:** ❌ NO - Pero útil para diagnosticar sobreajuste/subajuste.

**¿Qué muestra?**
- Cómo cambia el rendimiento del modelo según el tamaño del conjunto de entrenamiento
- **Brecha grande entre curvas:** Posible sobreajuste
- **Ambas curvas bajas:** Posible subajuste

**Parámetros:**
- **`cv=5`:** Número de folds para validación cruzada (📝 común: 5 o 10)
- **`scoring="f1_macro"`:** Métrica a evaluar (📝 opciones: "accuracy", "f1", "precision")
- **`train_sizes`:** Proporciones del dataset a usar (📝 np.linspace(0.1, 1.0, 5) = 10%, 32%, 55%, 77%, 100%)

In [ ]:
from sklearn.model_selection import learning_curve

# CALCULAR CURVA DE APRENDIZAJE
train_sizes, train_scores, valid_scores = learning_curve(
    estimator=DecisionTreeClassifier(max_depth=4, random_state=42),
    X=X, y=y,
    cv=5,                    # 📝 MODIFICAR: Número de folds
    scoring="f1_macro",      # 📝 MODIFICAR: Métrica a evaluar
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5),  # 📝 MODIFICAR: Tamaños a probar
    shuffle=True,
    random_state=42
)

# CALCULAR MEDIAS
train_mean = train_scores.mean(axis=1)
valid_mean = valid_scores.mean(axis=1)

# GRAFICAR
fig, ax = plt.subplots(figsize=(6, 4), dpi=120)
ax.plot(train_sizes, train_mean, marker="o", label="Entrenamiento")
ax.plot(train_sizes, valid_mean, marker="s", label="Validación (CV)")
ax.set_xlabel("Tamaño del conjunto de entrenamiento")
ax.set_ylabel("F1 macro")
ax.set_title("Curva de Aprendizaje — Árbol de Decisión")
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**📖 Interpretación:**
- **Curvas muy separadas:** El modelo está sobreajustando (demasiado complejo)
- **Curvas muy juntas pero bajas:** El modelo está subajustando (muy simple)
- **Curvas juntas y altas:** ¡Modelo bien balanceado! ✅

---
## 1️⃣5️⃣ PASO 15: Comparación de Criterios (Experimento Manual)

**Obligatorio:** ❌ NO - Solo para experimentación y aprendizaje.

**¿Qué hace?** Compara el rendimiento usando diferentes criterios de división.

**Útil para:** Entender el impacto de cada parámetro.

In [ ]:
# COMPARAR DIFERENTES CRITERIOS
criterios = ["gini", "entropy", "log_loss"]
resultados = []

for c in criterios:
    clf = DecisionTreeClassifier(
        criterion=c,
        max_depth=4,  # 📝 MODIFICAR: Profundidad fija para comparación justa
        random_state=42
    )
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    resultados.append((c, acc))

# MOSTRAR RESULTADOS
comparison_df = pd.DataFrame(resultados, columns=["criterion", "accuracy"])
comparison_df = comparison_df.sort_values("accuracy", ascending=False)
print("\n📊 Comparación de Criterios:")
print(comparison_df.to_string(index=False))

---
## 1️⃣6️⃣ PASO 16: Búsqueda Exhaustiva Manual (Experimento Avanzado)

**Obligatorio:** ❌ NO - Alternativa manual a GridSearchCV.

**¿Cuándo usar?** Si quieres ver TODAS las combinaciones de parámetros en una tabla.

**Parámetros adicionales:**
- **`min_samples_split`:** Mínimo de muestras para dividir un nodo (📝 por defecto: 2)
- **`min_samples_leaf`:** Mínimo de muestras en cada hoja (📝 por defecto: 1)
- **`max_features`:** Máximo de características a considerar (📝 None = todas)

In [ ]:
# BÚSQUEDA MANUAL EXHAUSTIVA
criterios = ["gini", "entropy", "log_loss"]
profundidades = [3, 4, 5, 6, 8, None]  # 📝 MODIFICAR: Profundidades a probar
resultados = []

print("⏳ Probando combinaciones...")
for c in criterios:
    for depth in profundidades:
        clf = DecisionTreeClassifier(
            criterion=c,
            max_depth=depth,
            min_samples_split=2,   # 📝 MODIFICAR: Ajustar según necesidad
            min_samples_leaf=1,    # 📝 MODIFICAR: Ajustar según necesidad
            max_features=None,     # 📝 MODIFICAR: None, "sqrt", "log2"
            random_state=42
        )
        clf.fit(X_train, y_train)
        acc = clf.score(X_test, y_test)
        resultados.append((c, depth, acc))

# CREAR TABLA DE RESULTADOS
results_df = pd.DataFrame(resultados, columns=["criterion", "max_depth", "accuracy"])
results_df = results_df.sort_values("accuracy", ascending=False)

print("\n✅ Resultados completos (ordenados por accuracy):")
print(results_df.to_string(index=False))

print(f"\n🏆 Mejor combinación: {results_df.iloc[0]['criterion']} + max_depth={results_df.iloc[0]['max_depth']} → {results_df.iloc[0]['accuracy']:.4f}")

---
## 1️⃣7️⃣ PASO 17: GridSearchCV con Más Parámetros (Búsqueda Avanzada)

**Obligatorio:** ❌ NO - Versión más completa de GridSearch.

**⚠️ ADVERTENCIA:** Probar muchos parámetros puede ser MUY LENTO.

**Parámetros adicionales explicados:**

### 🔧 `min_samples_split`
- Mínimo de muestras necesarias para dividir un nodo interno
- **Valores típicos:** 2 (por defecto), 5, 10, 20
- **Mayor valor:** Menos divisiones, árbol más simple

### 🔧 `min_samples_leaf`
- Mínimo de muestras necesarias en cada nodo hoja
- **Valores típicos:** 1 (por defecto), 2, 5, 10
- **Mayor valor:** Hojas más "gruesas", reduce sobreajuste

### 🔧 `max_features`
- Número máximo de características a considerar para cada división
- **Opciones:** None (todas), "sqrt" (raíz cuadrada), "log2" (logaritmo base 2)
- **Útil para:** Reducir correlación entre árboles en Random Forest

In [ ]:
# GRID SEARCH CON MUCHOS PARÁMETROS
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, None],
    # Descomentar para búsqueda más exhaustiva (¡más lento!):
    # "min_samples_split": [2, 5, 10],
    # "min_samples_leaf": [1, 2, 5],
    # "max_features": [None, "sqrt", "log2"]
}

grid = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    scoring="accuracy",  # 📝 MODIFICAR: "accuracy", "f1", "roc_auc"
    cv=5,                # 📝 MODIFICAR: Número de folds
    n_jobs=-1,
    verbose=1            # Mostrar progreso
)

print("⏳ Ejecutando búsqueda exhaustiva...")
grid.fit(X_train, y_train)

print("\n✅ Búsqueda completada")
print(f"\n🏆 Mejores parámetros: {grid.best_params_}")
print(f"📊 Mejor accuracy (CV): {grid.best_score_:.4f}")

---
## 1️⃣8️⃣ PASO 18: Modelo Final Definitivo

**Obligatorio:** ✅ SÍ - Entrenar el modelo final con los mejores parámetros encontrados.

**📝 IMPORTANTE:** Actualiza los parámetros con los valores de `grid.best_params_`.

In [ ]:
# MODELO FINAL CON LOS MEJORES PARÁMETROS
# 📝 MODIFICAR: Usa los valores que obtuviste del GridSearch anterior

clf_final = DecisionTreeClassifier(
    criterion="gini",    # 📝 Actualizar según best_params_
    max_depth=8,         # 📝 Actualizar según best_params_
    random_state=42
)

# ENTRENAR
clf_final.fit(X_train, y_train)

# EVALUAR
acc_final = clf_final.score(X_test, y_test)

print("="*50)
print("🎯 MODELO FINAL - RESULTADOS")
print("="*50)
print(f"Accuracy en test: {acc_final:.4f} ({acc_final*100:.2f}%)")
print(f"Profundidad del árbol: {clf_final.get_depth()}")
print(f"Número de hojas: {clf_final.get_n_leaves()}")
print("="*50)

---
## 🎓 RESUMEN Y GUÍA RÁPIDA

### ✅ Pasos OBLIGATORIOS (flujo mínimo):
1. Importar librerías
2. Cargar datos
3. Dividir datos (train/test)
4. Crear y entrenar modelo
5. Evaluar modelo

### ⚠️ Pasos OPCIONALES pero RECOMENDADOS:
- GridSearchCV (optimización de parámetros)
- Matriz de confusión y métricas
- Visualización del árbol
- Importancia de variables

### ❌ Pasos OPCIONALES según dataset:
- Variables dummy (solo si tienes categóricas)
- Escalado (no necesario para árboles)
- Graphviz (solo en Colab o con instalación local)

### 🔧 Parámetros principales para experimentar:

| Parámetro | Valores típicos | Efecto |
|-----------|----------------|--------|
| `criterion` | "gini", "entropy" | Método de división |
| `max_depth` | 3-10, None | Profundidad del árbol |
| `min_samples_split` | 2, 5, 10 | Control de divisiones |
| `min_samples_leaf` | 1, 2, 5 | Tamaño mínimo de hojas |
| `test_size` | 0.20-0.30 | Proporción de datos test |

---

## 💡 CONSEJOS PARA MODIFICAR Y EXPERIMENTAR:

1. **Para un nuevo dataset:**
   - Cambia el nombre del archivo CSV en el Paso 2
   - Cambia el nombre de la columna objetivo
   - Verifica si tienes variables categóricas (Paso 3)

2. **Para mejorar el modelo:**
   - Ejecuta GridSearchCV (Paso 7 o 17)
   - Usa los mejores parámetros encontrados
   - Prueba con diferentes métricas (accuracy, f1, precision)

3. **Para datasets pequeños (<1000 filas):**
   - Aumenta `test_size` a 0.30-0.40
   - Usa CV más alto (cv=10 en GridSearch)
   - Limita `max_depth` a 3-5 para evitar sobreajuste

4. **Para datasets grandes (>10000 filas):**
   - Reduce `test_size` a 0.10-0.20
   - Puedes probar árboles más profundos
   - Usa `n_jobs=-1` para paralelizar

---

**¡Listo para experimentar! 🚀**